In [1]:
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import os
from scipy import io
import numpy as np
import scipy
from PIL import Image, ImageOps
import cv2

In [2]:
load_dotenv(find_dotenv())
project_dir = Path(os.getenv("project_dir")).resolve()
output_filepath = project_dir / "data" / "processed"

In [3]:
archive = np.load(output_filepath / "vegfru_data.npz")
x_train = archive["x_train"]
y_train = archive["y_train"]
x_test = archive["x_test"]
y_test = archive["y_test"]

In [4]:
num_classes=292
target_size = 96
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

C:\Users\alexd\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [34]:
from sklearn.utils import shuffle

def preprocess_image(im):
    return cv2.resize(im, dsize=(target_size, target_size), interpolation=cv2.INTER_AREA)
    
def load_data_generator(x, y, batch_size=64):
    num_samples = x.shape[0]
    while 1:
        try:
            x, y = shuffle(x, y)
            for i in range(0, num_samples, batch_size):
                x_data = np.array([preprocess_image(x[i]) for i in range(i, i+batch_size)])
                y_data = y[i:i + batch_size]
            
                # convert to numpy array since this what keras required
                yield x_data, y_data
        except Exception as err:
            print(err)

In [35]:
# https://hackernoon.com/tf-serving-keras-mobilenetv2-632b8d92983c
from keras.applications.mobilenetv2 import MobileNetV2
from keras.layers import Dense, Input, Dropout
from keras.models import Model



def build_model():
    input_tensor = Input(shape=(target_size, target_size, 3))
    base_model = MobileNetV2(
        include_top=False,
        weights=None,
        input_tensor=input_tensor,
        input_shape=(target_size, target_size, 3),
        pooling='avg')

    for layer in base_model.layers:
        layer.trainable = True  
        
    op = Dense(256, activation='relu')(base_model.output)
    op = Dropout(.25)(op)
    
    output_tensor = Dense(num_classes, activation='softmax')(op)

    model = Model(inputs=input_tensor, outputs=output_tensor)


    return model

In [36]:
from keras.optimizers import Adam

model = build_model()
model.compile(optimizer=Adam(),
              loss="categorical_crossentropy",
              metrics=["categorical_accuracy"])

In [ ]:
batch_size = 16
train_generator = load_data_generator(x_train, y_train, batch_size=batch_size)

model.fit_generator(
    generator=train_generator,
    steps_per_epoch=20, #x_train.shape[0] // 64
    verbose=1,
    epochs=5)

Epoch 1/5
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
(16, 96, 96, 3)
 4/20 [=====>........................] - ETA: 7:52 - loss: 5.8588 - categorical_accuracy: 0.0000e+00(16, 96, 96, 3)
